In [ ]:
import tensorflow as tf
DEVICE = 'GPU'
numWorkers = len(tf.config.list_physical_devices('GPU'))
if numWorkers == 0:
    DEVICE = 'CPU'
    numWorkers = !cat /proc/cpuinfo | grep processor | wc -l
    numWorkers = int(numWorkers[0])
    
print(numWorkers)

In [ ]:
import random
import pandas
import numpy as np
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
import talos
import os
import math
from multiprocessing.pool import ThreadPool
from matplotlib import pyplot
import tensorflow_datasets as tfds
from ADAMApprox import AdamApprox
from dataset_loader import load_tf_dataset
from models_keras import get_model
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#global parameters for grid search over activation function
degree = 3
interval = 3
def reluApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.7146 + 1.5000*K.pow(x/interval,1)+0.8793*K.pow(x/interval,2)
        if interval == 5:
            return 0.7865 + 2.5000*K.pow(x/interval,1)+1.88*K.pow(x/interval,2)
        if interval == 7:
            return 0.9003 + 3.5000*K.pow(x/interval,1)+2.9013*K.pow(x/interval,2)
        if interval == 10:
            return 1.1155 + 5*K.pow(x/interval,1)+4.4003*K.pow(x/interval,2)
        if interval == 12:
            return 1.2751 + 6*K.pow(x/interval,1)+5.3803*K.pow(x/interval,2)
    if degree == 5:  
        if interval == 7:
            return 0.7521 + 3.5000*K.pow(x/interval,1)+4.3825*K.pow(x/interval,2)-1.7281*K.pow(x/interval,4)
        if interval == 20:
            return 1.3127 + 10*K.pow(x/interval,1)+15.7631*K.pow(x/interval,2)-7.6296*K.pow(x/interval,4)
def sigmoidApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.5 + 0.6997*K.pow(x/interval,1)-0.2649*K.pow(x/interval,3)
        if interval == 5:
            return 0.5 + 0.9917*K.pow(x/interval,1)-0.5592*K.pow(x/interval,3)
        if interval == 7:
            return 0.5 + 1.1511*K.pow(x/interval,1)-0.7517*K.pow(x/interval,3)
        if interval == 8:
            return 0.5 + 1.2010*K.pow(x/interval,1)-0.8156*K.pow(x/interval,2)
        if interval == 12:
            return 0.5 + 1.2384*K.pow(x/interval,1)-0.8647*K.pow(x/interval,2)
def tanApprox(x):
    if degree == 3: 
        if interval ==1:
            return 0.9797*K.pow(x/interval,1)-0.2268*K.pow(x/interval,3)        
        if interval ==2:
            return 1.7329*K.pow(x/interval,1)-0.8454*K.pow(x/interval,3)
        if interval == 3:
            return 2.1673*K.pow(x/interval,1)-1.3358*K.pow(x/interval,3)
        if interval == 5:
            return 2.5338*K.pow(x/interval,1)-1.8051*K.pow(x/interval,3)
        if interval == 7:
            return 2.6629*K.pow(x/interval,1) -1.9801*K.pow(x/interval,3)
        if interval == 12:
            return 2.7599*K.pow(x/interval,1)-2.1140*K.pow(x/interval,2)
    if degree == 12:
        print('ooopssss')

In [ ]:
def randomSplitClientsData(data, labels, ds_info):
    
    numParties = ds_info['num_clients']
    sample_height, sample_width, sample_channels = ds_info['sample_shape']
    num_classes = ds_info['num_classes']
    
    numSamplesPerClient = int(data.shape[0]/numParties)
    #print(numSamplesPerClient)
    clientsData = np.zeros((numParties,int(numSamplesPerClient),sample_height,sample_width,sample_channels))
    clientsDataLabels = np.zeros((numParties,int(numSamplesPerClient),num_classes))
    #print(numSamplesPerClient)
    ind = 0
    for i in range(numParties):
        clientsData[i] = data[ind:ind+numSamplesPerClient]
        clientsDataLabels[i]=labels[ind:ind+numSamplesPerClient]
        ind = ind+numSamplesPerClient
    return clientsData, clientsDataLabels

def prepare_data_for_X_clients(x_train, y_train, ds_info):
    clientsData, clientsDataLabels = randomSplitClientsData(x_train, y_train, ds_info)
    return clientsData, clientsDataLabels

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history, params):
    print('##########################################################')
    print(params)
    # plot loss
    pyplot.subplot(211)
    pyplot.title('MSE')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig(filename + '_plot.png')
    #pyplot.close()
    pyplot.show()
    print('##########################################################')

In [ ]:
def client_test_metrics(work):
    
        client_number, clientData, clientDataLabels, test_params, keep_range = work
    
        free = np.where(workers == 1)
        i = free[0][0]
        workers[i] = 0
    
        #Distribute load accross DEVICEs
        with tf.device(f"/{DEVICE}:{i%numWorkers}"):
            
            print(f"training on {DEVICE}: {i}")
            model = get_model(test_params, ds_info)
            
            optimizer = AdamApprox(keep_range=keep_range)
            print('Precision :', precision)
            optimizer.set_precision(precision)
            model.compile(loss="categorical_crossentropy", optimizer=optimizer, 
                          metrics=["accuracy", tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                          run_eagerly=keep_range)
            
            hist.append(model.fit(x=clientData,
                        y=clientDataLabels,
                        epochs=test_params['epochs'],
                        batch_size=test_params['batch_size'],
                        verbose=0))
            
            hist_precision.append(optimizer.get_range())
            
            print(f"client running on {DEVICE}: {i} finished")
            metrics = model.evaluate(x_test, y_test)
            metrics_res[client_number] = metrics
            
            workers[i] = 1
        return

def test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, test_params, keep_range):

    global metrics_res
    global workers
    
    metrics_res = np.zeros(numClients, dtype=object)   
    workers = np.ones(numWorkers)
                
    work = [(i, clientsData[i], clientsDataLabels[i], test_params, keep_range) for i in range(numClients)]
    
    with ThreadPool(len(workers)) as p:
        p.map(client_test_metrics, work)
    
    return metrics_res

In [ ]:
def run(params, ds, test_split, ds_info, runs):
    global interval
    interval = params['interval']
    
    numClients = ds_info['num_clients']
    
    (x_train, y_train) = ds
    
    global x_test
    global y_test
    (x_test, y_test) = test_split
    
    global hist
    global hist_precision
    
    hist = []
    hist_precision = []
    precisions = [0,1,2,4,6,8,10,12,'full']
    precisions = [0]

    precision_res = np.zeros(len(precisions) * runs, dtype=object)
    clientsData, clientsDataLabels = prepare_data_for_X_clients(x_train, y_train, ds_info)

    for i, p in enumerate(precisions):
        global precision
        precision = p
        keep_range = p == 'full'
        for r in range(runs):
            precision_res[i*runs+r] = test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, params, keep_range)
    print("\n\n")
    try:
        flat_list = [x for sublist in hist_precision for x in sublist]
        coeff_range = [min(flat_list), max(flat_list)]
        print("Coeff range (min, max):",coeff_range)
    except:
        print("No coeff range to show, if you want coeff range you should run the experiment with a 'full' precision")
        pass
    
    
    print("Loss, accuracy, precision, recall")
    res = np.zeros(4,dtype=float)
    for i,p in enumerate(precision_res):
        res[0] += p[0][0]
        res[1] += p[0][1]
        res[2] += p[0][2]
        res[3] += p[0][3]
        if i % runs == (runs-1):
            print(res/runs)
            print()
            res = np.zeros(4,dtype=float)

In [ ]:
dataset_name = 'svhn_cropped'
experiment_name = f"{dataset_name}_iid"
runs = 2

params = dict(learn_rate=0.3,
              batch_size=16,
              momentum=0.91,
              epochs=12,
              act_fn=reluApprox,
              interval=7)

ds, test_split, ds_info = load_tf_dataset(dataset_name=dataset_name, decentralized=True, display=True)
ds_info['num_clients'] = 1
run(params, ds, test_split, ds_info, runs)